### Byte Pair Encoding

In [1]:
#Install tiktoken library
#In terminal: pip install tiktoken
#Check Version
from importlib.metadata import version
import tiktoken
print("tiktoken version:",version("tiktoken"))

tiktoken version: 0.12.0
